In [1]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *


training = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "sentimentdataset")

validation = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "validation")
testing = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "testing")

#remove nonASCIICharacters
training["Text"] = training.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)
validation["Text"] = validation.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)
testing["Text"] = testing.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)






In [2]:
def to_class(s):
    match s:
        case "Positive":
            return 2
        case "Negative":
            return 1
        case "Neutral":
            return 0
    

In [3]:
training["SimpleSent"] = training.apply(lambda x: to_class(x["SimpleSent"]), axis=1)
testing["SimpleSent"] = testing.apply(lambda x: to_class(x["SimpleSent"]), axis=1)
validation["SimpleSent"] = validation.apply(lambda x: to_class(x["SimpleSent"]), axis=1)

In [4]:
toDrop = ["Timestamp", "User", "Platform", "Retweets", "Likes", "Country", "Year", "Month", "Day", "Hour"]

training = training.drop(toDrop, axis=1)
testing = testing.drop(toDrop, axis=1)
validation = validation.drop(toDrop, axis=1)


In [5]:
import re
def split_words(xs):
    empties = []
    empty = "" 
    for i in xs:
        words = re.split(r"([A-Z])", i)
        words = words[1::]
        for i in range(0, len(words) -1, 2):
            empty += words[i] + words[i + 1]
            empty += " "
        empties.append(empty.rstrip())
    return empties

In [6]:
from textblob import *
#process Hashtages
training["proc_hash"] = training.apply(lambda x: x["Hashtags"][1::], axis=1)
training["proc_hash"] = training.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
training["proc_hash"] = training.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
training["Hashtags"] = training["proc_hash"]
training = training.drop("proc_hash", axis=1)

testing["proc_hash"] = testing.apply(lambda x: x["Hashtags"][1::], axis=1)
testing["proc_hash"] = testing.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
testing["proc_hash"] = testing.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
testing["Hashtags"] = testing["proc_hash"]
testing = testing.drop("proc_hash", axis=1)

validation["proc_hash"] = validation.apply(lambda x: x["Hashtags"][1::], axis=1)
validation["proc_hash"] = validation.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
validation["proc_hash"] = validation.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
validation["Hashtags"] = validation["proc_hash"]
validation = validation.drop("proc_hash", axis=1)

validation






,Text,Sentiment,SimpleSent,Hashtags
0,"After a series of defeats, the soccer team fac...",Disappointment,1,Disappointment and Disappointment Soccer Defeats
1,"In the tennis tournament, a highly anticipated...",Frustration,1,Frustration and Frustration Tennis Setback
2,"Facing a defeat in the championship, the boxer...",Reflection,0,Reflection and Reflection Boxing Defeat
3,"In the midst of a cycling race, a tire blowout...",Obstacle,0,Obstacle and Obstacle Cycling Frustration
4,The gymnast's unexpected fall during a routine...,Sympathy,2,Sympathy and Sympathy Gymnastics Fall
...,...,...,...,...
86,"Organized a community painting event, turning ...",Joy,2,Community Art and Community Art Senior Creativity
87,Hosted a 'memory lane' evening with old friend...,Gratitude,2,Friendship Adventures and Friendship Adventure...
88,"Joined a seniors' astronomy club, stargazing a...",Curiosity,0,Celestial Wonders and Celestial Wonders Senior...
89,"Attended a local jazz festival, tapping toes t...",Joy,2,Timeless Tunes and Timeless Tunes Senior Jazz


In [7]:


training["text_tb_pol"] = training.apply(lambda x: TextBlob(x["Text"]).polarity, axis=1)
training["text_tb_sub"] = training.apply(lambda x: TextBlob(x["Text"]).subjectivity, axis=1)

training["hash_tb_pol"] = training.apply(lambda x: TextBlob(x["Hashtags"]).polarity, axis=1)
training["hash_tb_sub"] = training.apply(lambda x: TextBlob(x["Hashtags"]).subjectivity, axis=1)




In [8]:
analyzer = SentimentIntensityAnalyzer()

training["text_vader_scores"] = training.apply(lambda x: analyzer.polarity_scores(x["Text"]), axis=1)
training["text_vader_neg"] = training.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
training["text_vader_neu"] = training.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
training["text_vader_pos"] = training.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
training["text_vader_comp"] = training.apply(lambda x: x["text_vader_scores"]["compound"], axis=1)
training = training.drop(["text_vader_scores"], axis=1)

training["hash_vader_scores"] = training.apply(lambda x: analyzer.polarity_scores(x["Hashtags"]), axis=1)
training["hash_vader_neg"] = training.apply(lambda x: x["hash_vader_scores"]["neg"], axis=1)
training["hash_vader_neu"] = training.apply(lambda x: x["hash_vader_scores"]["neu"], axis=1)
training["hash_vader_pos"] = training.apply(lambda x: x["hash_vader_scores"]["pos"], axis=1)
training["hash_vader_comp"] = training.apply(lambda x: x["hash_vader_scores"]["compound"], axis=1)
training = training.drop(["hash_vader_scores"], axis=1)





In [ ]:
training

In [9]:
'''
    Begin Neural Net
'''

classes = training["SimpleSent"].to_numpy()
#features = training[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()

'''text only'''
features = training[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(4, 1)),
  tf.keras.layers.Dense(4, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])



model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=1)
model.fit(x_train, y_train, epochs=100, verbose=2)
model.save("TWEETS_NN.keras")

Epoch 1/100


c:\Users\desmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 - 1s - 51ms/step - accuracy: 0.2942 - loss: 1.1340
Epoch 2/100
20/20 - 0s - 3ms/step - accuracy: 0.4019 - loss: 1.0905
Epoch 3/100
20/20 - 0s - 3ms/step - accuracy: 0.4871 - loss: 1.0516
Epoch 4/100
20/20 - 0s - 3ms/step - accuracy: 0.5514 - loss: 1.0166
Epoch 5/100
20/20 - 0s - 4ms/step - accuracy: 0.6206 - loss: 0.9863
Epoch 6/100
20/20 - 0s - 4ms/step - accuracy: 0.6045 - loss: 0.9600
Epoch 7/100
20/20 - 0s - 3ms/step - accuracy: 0.6013 - loss: 0.9360
Epoch 8/100
20/20 - 0s - 3ms/step - accuracy: 0.6029 - loss: 0.9150
Epoch 9/100
20/20 - 0s - 3ms/step - accuracy: 0.6029 - loss: 0.8965
Epoch 10/100
20/20 - 0s - 3ms/step - accuracy: 0.6045 - loss: 0.8794
Epoch 11/100
20/20 - 0s - 8ms/step - accuracy: 0.6061 - loss: 0.8648
Epoch 12/100
20/20 - 0s - 3ms/step - accuracy: 0.6077 - loss: 0.8514
Epoch 13/100
20/20 - 0s - 3ms/step - accuracy: 0.6109 - loss: 0.8386
Epoch 14/100
20/20 - 0s - 3ms/step - accuracy: 0.6174 - loss: 0.8269
Epoch 15/100
20/20 - 0s - 3ms/step - accuracy: 0.6270 

In [ ]:

model_loss1, model_acc1 = model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = model.evaluate(x_test,  y_test, verbose=2)
print(f"Train / Test Accuracy: {model_acc1*100:.1f}% / {model_acc2*100:.1f}%")

In [ ]:
'''
from sklearn.metrics import confusion_matrix
matrices = []
for i in range(100):
  model = tf.keras.models.load_model("SANN.keras")
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
  model.fit(x_train, y_train, epochs=100)

  y_pred = model.predict(x_test)
  class_preds = np.argmax(y_pred, axis=-1)
  matrices.append(confusion_matrix(y_test, class_preds))

mean_matrix = np.mean(np.array(matrices), axis=0)
mean_matrix
'''

